![rustronomy_dark_banner](https://github.com/smups/rustronomy/blob/main/logos/Rustronomy-fits_github_banner_dark.png?raw=true#gh-light-mode-only)
# Example 1 - reading a FITS file

Let's start by importing some dependancies:

In [2]:
:dep rustronomy-fits = {git = "https://github.com/smups/rustronomy"}
:dep ndarray = {ndarray = 0.15, features = ["blas", "approx", "std"]}
:dep dirs = {dirs = 4}

In [12]:
use rustronomy_fits as rsf;
use ndarray as nd;
use dirs;

### 1.1 - displaying the structure of a FITS file
When using Rustronomy-fits, it's often important to know what kind of data the FITS file contains _before_ you try to access any of the data contained within. This has to do with Rust being statically typed: you cant just ask for "data contained within HDU0" without knowing the type of the data.

Luckily, we can simply print the structure of a FITS file.

_Note: in this example we are using an image taken by the Hubble Space Telescope.
The required file is provided in the resources folder on github_

In [13]:
use std::fs;
use std::path;

//We'll be using one of the FITS files from the tests folder
let mut path = dirs::home_dir().unwrap();
path.push("Desktop/code/rustronomy/rustronomy/rustronomy-fits/resources/Hubble_NICMOS.fits");

//open and print the file (we need the mut later on)
let mut fits = rsf::Fits::open(&path)?;
println!("{fits}");


>=================================<|FITS File|>=================================
>Total File size in FITS blocks: 416
>Number of Header-Data-Units: 6
>Contents:
>-------------------------------------------------------------------------------
>  [HDU #0]
>  Total HDU size in FITS blocks: 7
>    [Header] - size: 7, #records: 167
>    [Data] - (NO_DATA)
>-------------------------------------------------------------------------------
>  [HDU #1]
>  Total HDU size in FITS blocks: 103
>    [Header] - size: 4, #records: 80
>    [Data] - (IMAGE) datatype: f32, shape: (270,263), size: 99
>-------------------------------------------------------------------------------
>  [HDU #2]
>  Total HDU size in FITS blocks: 101
>    [Header] - size: 2, #records: 35
>    [Data] - (IMAGE) datatype: f32, shape: (270,263), size: 99
>-------------------------------------------------------------------------------
>  [HDU #3]
>  Total HDU size in FITS blocks: 52
>    [Header] - size: 2, #records: 35
>    [Data] 

from the print-out we can see that we have a `f32` Image in header-data-unit #2.
We can get more information about the Image by printing its header:

In [14]:
println!("{}", fits.get_hdu(2).unwrap().get_header());

>================================<|FITS Header|>================================
>Size in FITS blocks: 2
>  [INHERIT] - T //inherit the primary header
>  [CTYPE2] - DEC--TAN //the coordinate type for the second axis
>  [STDCFFF] - F / ST DDF fill present (T/F)
>  [CRVAL1] - 1.826361425552E+02 //first axis value at reference pixel
>  [ORIGIN] - NOAO-IRAF FITS Image Kernel December 2001 //FITS file originator
>  [EXTNAME] - ERR //Extension name
>  [CD2_1] - -2.09168E-05 //partial of second axis coordinate w.r.t. x
>  [CTYPE1] - RA---TAN //the coordinate type for the first axis
>  [CD1_2] - -2.07292E-05 //partial of first axis coordinate w.r.t. y
>  [ERRCNT] - 0 //number of segments containing errors
>  [CRPIX2] - 1.3200000E+02 //y-coordinate of reference pixel
>  [CD1_1] - 2.86146E-06 //partial of first axis coordinate w.r.t. x
>  [STDCFFP] - x5569 //ST DDF fill pattern (hex)
>  [EXTVER] - 1 //Extension version
>  [ROOTNAME] - N4HK12010
>  [EXPNAME] - N4HK12010 //exposure identifier
>  [

### 1.2 - loading the data as a ndarray

In general, the `get_xyz()` methods in rustronomy-fits return references (unless the return type is primitive). To obtain an owned version of the data and header contained within the hdu, we have to use a `remove_hdu()` method to obtain an owned version of the hdu, which we then deconstruct using the `to_parts()` method:

In [15]:
let (header, data) = fits.remove_hdu(2).unwrap().to_parts();

Our next task is to convert the data that we received to something usefull. FITS files can contain various data containers, such as Images and Tables. These are represented in rsf with the `Extension` enum. `data` is one of the variants of this extension enum. In our case, we know that we have a FITS Image, which is represented as the `Image` variant. 

The actual image is contained within the enum variant. We can turn it into a ndarray by calling the `as_owned_..._array` method where we replace ... with the datatype of the image, in our case: `f32`:

In [16]:
let array = match data.unwrap() {
    rsf::Extension::Image(img) => img.as_owned_f32_array()?,
    _ => panic!()
};
println!("{array}");

[[0, 0, 0, 0, 0, ..., 0.14516334, 0.1677333, 0.1519012, 0.17424014, 0.18138014],
 [0, 0, 0, 0, 0, ..., 0.14382742, 0.13788727, 0.14309902, 0.18218201, 0.1548093],
 [0, 0.25458828, 0.19007798, 0.19276433, 0.16891634, ..., 0.15199432, 0.14267182, 0.13510334, 0.140643, 0.1438491],
 [0.23901846, 0.24790978, 0.17884323, 0.19360472, 0.17666389, ..., 0.14283721, 0.15122496, 0.14378895, 0.1394441, 0.16420925],
 [0.22904848, 0.22937974, 0.24271114, 0.16900842, 0.15040079, ..., 0.1495607, 0.14531259, 0.14317311, 0.15108435, 0.16691282],
 ...,
 [0, 0.24676378, 0.23727417, 0.23779844, 0.2573741, ..., 0.21935812, 0.21897647, 0.26512703, 0.24180707, 0.25430027],
 [0, 0.22375634, 0.23354273, 0.2748666, 0.20880164, ..., 0.23301119, 0.23804893, 0.23298545, 0.23288651, 0.21402515],
 [0, 0.274768, 0.24180919, 0.25705317, 0.19924851, ..., 0.24541865, 0.25179303, 0.24744774, 0.24135269, 0.27544743],
 [0, 0.2952811, 0.26060498, 0.27895373, 0.22497027, ..., 0.20358875, 0.22370559, 0.22956917, 0.2574325, 0.26